<p style="font-size:22pt;font-family:'Ubuntu';">Data Cleansing (uncompleted)</p>

<br>
[Data cleansing](https://en.wikipedia.org/wiki/Data_cleansing) is integral part of any project where big sets of data come into play. In this document I will try to implement some basic data cleansing functions in python.
<br>

The data is in csv format and contains nearly two thousand records. Some rows in each column have been damaged, so the data is far from being consistent. 

<br>
modules used in this worksheet includes **pandas** dataframe to work on the data, some python build in libraries and **numpy**





In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv('project-dirty.csv', delimiter=';')
dataframe.count;

In [ ]:
""" clean the sixth column which contains unconsistent money value format data. """

In [ ]:
import numpy as np

def equalize(money):
    """ replace all others delimeters to dot """
    
    return money.replace(',', '.').replace('`', '.').replace("’", ".")

def transform(money):
    """ transform invalid money format to vaild one """
    
    multiplier = 1
    money = money.replace('zł', '').strip()
    if money.find('tys') != -1:
        money = money.replace('tys', '').strip()
        multiplier = 1000
        
    if money.find('mln') != -1:
        money = money.replace('mln', '').strip()
        multiplier = 1000000
    
    money = "".join(money.split())
    
    splitted = money.split('.')
    if len(splitted)>1 and len(splitted[-1])==2:
        money = "".join(splitted[:-1]) + "."+splitted[-1]
    else:
        money = "".join(splitted)
    return int(float(money) * multiplier)

def launder_money(amount):
    """ apply all changes"""
    
    if amount is np.nan:
        return 0
    else:
        base_string = str(amount).strip()
        equalized_string = equalize(base_string)
        transformed_string = transform(equalized_string)
        return transformed_string

dataframe[dataframe.columns[5]] = dataframe[dataframe.columns[5]].apply(launder_money)

In [ ]:
dataframe = dataframe.replace(r'\n',' ', regex=True) 
dataframe[dataframe.columns[4]] = dataframe[dataframe.columns[4]].str.replace('•', '')

In [ ]:
""" clean the third column which contains unconsistent datetime data """

In [ ]:
def date_deterimner(date):
    """ get number of years where row is not in date format"""
    if date is np.nan:
        return 0
    return int(str(date)[0])

def new_cleaned_closed_date(start_close_dates):
    def is_date(string):
        if string is np.nan:
            return False
        from dateutil.parser import parse
        try: 
            parse(string)
            return True
        except ValueError:
            return False
        
    def my_uber_date_fmt(date, formats):
        for candidate in formats:
            try:
                return datetime.datetime.strptime(date, candidate)
            except:
                pass
            
        raise ValueError('Could not perform formatting with supplied formats ' + str(date))
    
    #get format date style based on start_date column determined by date_determiner func
    import datetime
    from dateutil.relativedelta import relativedelta
    
    start_date, close_date = start_close_dates
    
    if start_date is np.nan:
        return [str(np.nan), str(np.nan)]

    date_formats = ["%Y-%m-%d", "%y-%m-%d", 
                    "%Y %m %d", "%y %m %d",
                    "%d.%m.%Y", "%d.%m.%y"]
    start_date = my_uber_date_fmt(start_date, date_formats)
    
    if not is_date(close_date):
        close_date = start_date + relativedelta(years=date_deterimner(close_date))
        close_date = str(close_date)[:10]

    close_date = my_uber_date_fmt(close_date, date_formats)
    return [str(start_date)[:10], str(close_date)[:10]]


In [ ]:
dataframe[dataframe.columns[2:4]] = dataframe[dataframe.columns[2:4]].apply(new_cleaned_closed_date, raw=True, axis=1)


In [ ]:
from openpyxl import load_workbook
workbook = load_workbook('names.xlsx')
worksheet = workbook.active
head_row, *rows = worksheet.values
names_data = pd.DataFrame(rows, columns=head_row)
name_to_sex = names_data[['Imię', 'Płeć']]
name_to_sex;